#### 1. Giải nén

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os

# Thư mục project trên máy
PROJECT_DIR = r"D:\Code\PythonProject\Advanced\multi_task_ct"

# Thư mục chứa data gốc (các .zip, .nii.gz)
INPUT_DIR = r"D:\Code\PythonProject\Advanced\datasets"

# Thư mục muốn lưu dữ liệu đã giải nén (.nii, thư mục từ zip)
OUTPUT_DIR = os.path.join(PROJECT_DIR, "data", "raw")

RAW_ROOT = os.path.join(
    PROJECT_DIR,
    "data",
    "raw",
    "covid19_normal"
)

OUT_ROOT = os.path.join(
    PROJECT_DIR,
    "data",
    "processed",
    "covid_normal"
)

print("PROJECT_DIR:", PROJECT_DIR)
print("INPUT_DIR  :", INPUT_DIR)
print("OUTPUT_DIR :", OUTPUT_DIR)
print("RAW_ROOT   :", RAW_ROOT)
print("OUT_ROOT   :", OUT_ROOT)

PROJECT_DIR: D:\Code\PythonProject\Advanced\multi_task_ct
INPUT_DIR  : D:\Code\PythonProject\Advanced\datasets
OUTPUT_DIR : D:\Code\PythonProject\Advanced\multi_task_ct\data\raw
RAW_ROOT   : D:\Code\PythonProject\Advanced\multi_task_ct\data\raw\covid19_normal
OUT_ROOT   : D:\Code\PythonProject\Advanced\multi_task_ct\data\processed\covid_normal


In [6]:
!python "{PROJECT_DIR}/src/preprocess/extract_all_data.py" --input_dir "{INPUT_DIR}" --output_dir "{OUTPUT_DIR}"

  🚀 BẮT ĐẦU CHUẨN BỊ DỮ LIỆU NIFTI
📥 INPUT  (read-only): D:\Code\PythonProject\Advanced\datasets
📤 OUTPUT (working)  : D:\Code\PythonProject\Advanced\multi_task_ct\data\raw

📁 Quét .zip trong INPUT (chỉ đọc, không xóa): D:\Code\PythonProject\Advanced\datasets
🔍 Tìm thấy 2 file .zip trong INPUT_DIR

📦 Giải nén (INPUT -> OUTPUT): D:\Code\PythonProject\Advanced\datasets\covid19_lung_infection.zip
   ➜ Vào: D:\Code\PythonProject\Advanced\multi_task_ct\data\raw\.\covid19_lung_infection

📦 Giải nén (INPUT -> OUTPUT): D:\Code\PythonProject\Advanced\datasets\covid19_normal.zip
   ➜ Vào: D:\Code\PythonProject\Advanced\multi_task_ct\data\raw\.\covid19_normal

✅ Hoàn thành bước giải nén .zip từ INPUT sang OUTPUT.

📁 Quét .nii.gz trong INPUT (chỉ đọc, không xóa): D:\Code\PythonProject\Advanced\datasets
🔍 Tìm thấy 0 file .nii.gz trong INPUT_DIR

✅ Hoàn thành chuyển .nii.gz từ INPUT sang OUTPUT.


🔁 Vòng 1: tìm thấy 6 file .zip trong OUTPUT

📦 Giải nén (OUTPUT): D:\Code\PythonProject\Advanced\multi_

#### 2. Chạy tách nền

In [ ]:
!pip install SimpleITK lungmask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 73.6 MB/s eta 0:00:00


In [ ]:
!python "{PROJECT_DIR}/src/preprocess/covid_normal_convert3d_to_2d.py" --raw_root "{RAW_ROOT}" --out_root "{OUT_ROOT}"

Tổng số case COVID : 230
Tổng số case NORMAL: 149

📄 Đã tồn tại meta.csv, đọc để lấy danh sách case_id đã xử lý: /content/drive/MyDrive/multi_task_ct/data/processed/covid_normal/meta.csv
🔁 Đã có 234 case_id trong meta.csv → sẽ SKIP những case này.

[SKIP] COVID p100 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p101 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p102 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p103 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p104 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p105 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p106 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p107 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p108 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p109 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p10 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p110 đã có trong meta.csv → bỏ qua (theo CSV).
[SKIP] COVID p111 đã có trong me

#### 3. Train model Unet multi-task

In [ ]:
import os, sys

project_root = "/content/drive/MyDrive/multi_task_ct"
os.chdir(project_root)

if project_root not in sys.path:
    sys.path.append(project_root)

print("Current dir:", os.getcwd())

Current dir: /content/drive/MyDrive/multi_task_ct


In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))


CUDA available: False


In [ ]:
!python -m src.data.make_splits \
  --data_root data/processed/covid_normal \
  --seed 42


Tổng số case: 301
Train case: 210, slices: 7209
Val   case: 45, slices: 1670
Test  case: 46, slices: 1758
Đã lưu splits_by_case vào: data/processed/covid_normal/splits


In [ ]:
%cd /content/drive/MyDrive/multi_task_ct
import sys
sys.path.append("/content/drive/MyDrive/multi_task_ct")

/content/drive/MyDrive/multi_task_ct


In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

print("=== Step 1: Copy project to /content for fast training ===")

# Xóa bản local cũ (nếu có)
!rm -rf /content/multi_task_ct_local

# Tạo thư mục project local
!mkdir -p /content/multi_task_ct_local

# Copy folder src
!cp -r /content/drive/MyDrive/multi_task_ct/src /content/multi_task_ct_local/

# Copy dataset processed (chỉ covid_normal)
!mkdir -p /content/multi_task_ct_local/data/processed
!cp -r /content/drive/MyDrive/multi_task_ct/data/processed/covid_normal /content/multi_task_ct_local/data/processed/

# Tạo sẵn checkpoints & logs (trống cũng được)
!mkdir -p /content/multi_task_ct_local/checkpoints
!mkdir -p /content/multi_task_ct_local/logs

print("✅ Done Step 1: Project copied to /content/multi_task_ct_local")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== Step 1: Copy project to /content for fast training ===


In [2]:
import os

print("=== Step 2: Fix utils folder (rename untils → utils) ===")

src_path = "/content/multi_task_ct_local/src"

# Nếu có thư mục untils thì đổi tên sang utils
if os.path.isdir(f"{src_path}/untils"):
    !mv /content/multi_task_ct_local/src/untils /content/multi_task_ct_local/src/utils

# Nếu chưa có utils thì tạo
if not os.path.isdir(f"{src_path}/utils"):
    !mkdir /content/multi_task_ct_local/src/utils

# Nếu chưa có metrics.py thì tạo file default
metrics_file = f"{src_path}/utils/metrics.py"
if not os.path.isfile(metrics_file):
    print("⚠️ metrics.py not found → creating a default metrics file")
    metrics_code = """
import torch

def dice_coefficient(pred, target, threshold=0.5):
    pred_bin = (pred > threshold).float()
    target_bin = (target > threshold).float()
    inter = (pred_bin * target_bin).sum(dim=(1,2,3))
    union = pred_bin.sum(dim=(1,2,3)) + target_bin.sum(dim=(1,2,3))
    dice = (2*inter + 1e-7) / (union + 1e-7)
    return dice.mean().item()

def iou_score(pred, target, threshold=0.5):
    pred_bin = (pred > threshold).float()
    target_bin = (target > threshold).float()
    inter = (pred_bin * target_bin).sum(dim=(1,2,3))
    union = pred_bin.sum(dim=(1,2,3)) + target_bin.sum(dim=(1,2,3)) - inter
    iou = (inter + 1e-7) / (union + 1e-7)
    return iou.mean().item()

def classification_metrics(logits, labels):
    preds = torch.argmax(logits, dim=1)
    correct = (preds == labels).float().mean().item()
    return correct
"""
    with open(metrics_file, "w") as f:
        f.write(metrics_code)

print("✅ Done Step 2")

=== Step 2: Fix utils folder (rename untils → utils) ===
✅ Done Step 2


In [3]:
import fileinput

print("=== Step 3: Patch DataLoader num_workers=2 ===")

train_file = "/content/multi_task_ct_local/src/train/train_unet_multitask.py"

# Thay tất cả 'num_workers=0' thành 'num_workers=2'
for line in fileinput.input(train_file, inplace=True):
    print(line.replace("num_workers=0", "num_workers=2"), end='')

print("✅ Updated DataLoader with num_workers=2")

=== Step 3: Patch DataLoader num_workers=2 ===
✅ Updated DataLoader with num_workers=2


In [4]:
import os, sys

print("=== Step 4: Move to project local directory ===")

%cd /content/multi_task_ct_local

if "/content/multi_task_ct_local" not in sys.path:
    sys.path.append("/content/multi_task_ct_local")

print("Current working directory:", os.getcwd())
print("Project structure:")
!ls -R .
print("✅ Done Step 4")

=== Step 4: Move to project local directory ===
/content/multi_task_ct_local
Current working directory: /content/multi_task_ct_local
Project structure:
.:
checkpoints  data  logs  src

./checkpoints:

./data:
processed

./data/processed:
covid_normal

./data/processed/covid_normal:
images	masks

./data/processed/covid_normal/images:
img  img_roi

./data/processed/covid_normal/images/img:
100_z006.png  17_z018.png  66_z020.png	  p122_z017.png  p41_z034.png
100_z007.png  17_z019.png  66_z021.png	  p122_z018.png  p41_z035.png
100_z008.png  17_z020.png  66_z022.png	  p122_z019.png  p41_z036.png
100_z009.png  17_z021.png  66_z023.png	  p122_z020.png  p41_z037.png
100_z010.png  17_z022.png  66_z024.png	  p122_z021.png  p41_z038.png
100_z011.png  17_z023.png  66_z025.png	  p122_z022.png  p41_z039.png
100_z012.png  17_z024.png  66_z026.png	  p122_z023.png  p41_z040.png
100_z013.png  17_z025.png  66_z027.png	  p122_z024.png  p41_z041.png
100_z014.png  17_z026.png  66_z028.png	  p122_z025.png  p

In [5]:
print("=== Step 5: TRAINING 1 EPOCH ===")

# EPOCHS=1 để test pipeline
!EPOCHS=1 python -m src.train.train_unet_multitask

print("✅ Done Step 5: Training finished (1 epoch)")

=== Step 5: TRAINING 1 EPOCH ===
Device: cpu
Splits dir: /content/multi_task_ct_local/data/processed/covid_normal/splits
Num epochs: 1
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/multi_task_ct_local/src/train/train_unet_multitask.py", line 301, in <module>
    main()
  File "/content/multi_task_ct_local/src/train/train_unet_multitask.py", line 164, in main
    train_set = LungCTMultiTaskDataset(
                ^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/multi_task_ct_local/src/data/dataset_ct.py", line 33, in __init__
    with open(meta_csv, "r", newline="") as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/multi_task_ct_local/data/processed/covid_normal/meta.csv'
✅ Done Step 5: Training finished (1 epoch)


In [ ]:
print("=== Step 6: Copy results back to Drive ===")

# Copy checkpoints + logs
!cp -r /content/multi_task_ct_local/checkpoints /content/drive/MyDrive/multi_task_ct/
!cp -r /content/multi_task_ct_local/logs /content/drive/MyDrive/multi_task_ct/

# Copy hình training curves nếu có
if os.path.exists("/content/multi_task_ct_local/training_curves_multitask.png"):
    !cp /content/multi_task_ct_local/training_curves_multitask.png /content/drive/MyDrive/multi_task_ct/

print("🎉 DONE! Checkpoints, logs, curves copied to /MyDrive/multi_task_ct")

#### 4. Train model Unet

In [1]:
import os, sys

# Lấy đường dẫn tới folder multi_task_ct (project root)
PROJECT_ROOT = r"D:\Code\PythonProject\Advanced\multi_task_ct"

# Thêm vào sys.path
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT added:", PROJECT_ROOT)

PROJECT_ROOT added: D:\Code\PythonProject\Advanced\multi_task_ct


In [2]:
import os, sys
from pathlib import Path

# Tìm PROJECT_ROOT (thư mục chứa folder src)
cwd = Path().resolve()
project_root = None
for p in [cwd, *cwd.parents]:
    if (p / "src").is_dir():
        project_root = p
        break

if project_root is None:
    raise RuntimeError("Không tìm được folder 'src' ở các thư mục cha.")

project_root = str(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PROJECT_ROOT =", project_root)
print("sys.path[0] =", sys.path[0])

PROJECT_ROOT = D:\Code\PythonProject\Advanced\multi_task_ct
sys.path[0] = D:\Code\PythonProject\Advanced\multi_task_ct


In [4]:
from src.train.train_unet import main

# có thể chỉnh env EPOCHS trong notebook:
import os
os.environ["EPOCHS"] = "20"   # ví dụ train thử 2 epoch

main()   # chạy training

Device: cuda
Splits dir: D:\Code\PythonProject\Advanced\multi_task_ct\data\processed\covid_normal\splits
Num epochs: 20
Model: UNet
Model parameters: 31,042,369

===== Epoch 1/20 =====


Val 1: 100%|██████████████████████████████████████████████████████| 902/902 [03:06<00:00,  4.82it/s]


Train - loss: 0.5653, dice: 0.4271, iou: 0.3717
Val   - loss: 0.4883, dice: 0.5713, iou: 0.5205, lr: 1.000000e-04
  -> Saved BEST model (val dice=0.5713)

===== Epoch 2/20 =====


Val 2: 100%|██████████████████████████████████████████████████████| 902/902 [03:03<00:00,  4.90it/s]


Train - loss: 0.3703, dice: 0.5516, iou: 0.4936
Val   - loss: 0.4076, dice: 0.6452, iou: 0.5875, lr: 1.000000e-04
  -> Saved BEST model (val dice=0.6452)

===== Epoch 3/20 =====


...

Train - loss: 0.1838, dice: 0.7639, iou: 0.7078
Val   - loss: 0.3147, dice: 0.7875, iou: 0.7325, lr: 1.000000e-04
  -> Saved BEST model (val dice=0.7875)

===== Epoch 20/20 =====


Val 20: 100%|█████████████████████████████████████████████████████| 902/902 [03:03<00:00,  4.92it/s]


Train - loss: 0.1823, dice: 0.7717, iou: 0.7156
Val   - loss: 0.3023, dice: 0.7942, iou: 0.7383, lr: 1.000000e-04
  -> Saved BEST model (val dice=0.7942)

===== Evaluate BEST model on TEST =====


Test 0: 100%|█████████████████████████████████████████████████████| 220/220 [00:52<00:00,  4.21it/s]

Test - loss: 0.3136, dice: 0.7607, iou: 0.7064
